In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import fitparse as fp
import csv
import pytz
import datetime
from pathlib import Path
import os

In [4]:
def print_record(data):
    """
    test method to print out the data in a record
    """
    for record in data:
        for field in record:
            if field.units:
                print (" * %s: %s %s" % (
                    field.name, field.value, field.units))
            else:
                print (" * %s: %s" % (field.name, field.value))

In [5]:
def get_time(record):
    """
    extract the time from data. Note for heart rate data the timestamp is int(16) up to 65536 ~ 18hours
    This will overflow and needs normalised so it wont give a crazy result on overflow.

    Every now and then there will be a timestamp giving the right date and time. 
    Heart rate data is timestamped on a rolling seconds counter. Need to set the first timestamp_16 to the
    global time then work out the next time based on the difference between timestamp_16s adding that to global time
    """
    new_timestamp_16 = None
    
    # difference between successive timestamp_16
    delta_diff = 0
    global timestamp_16
    global current_timestamp

    for r in record:
        if r.field is not None:
            if r.name == 'timestamp_16':
                new_timestamp_16 = r.value

    # if the timestamp is none then set it to the global time
    if new_timestamp_16 is not None:
        if timestamp_16 is None:
            timestamp_16 = new_timestamp_16
        
        delta_diff = new_timestamp_16 - timestamp_16
        
        # deal with overflow
        if delta_diff < 0:
            delta_diff += 65536
        
        current_timestamp += datetime.timedelta(seconds=delta_diff)
        timestamp_16 = new_timestamp_16
        #print(current_timestamp)
        return current_timestamp

In [6]:
def get_heartrate(record):
    """
    Return the heart rate from the record
    """
    for r in record:
        if r.name == 'heart_rate':
            return r.value

In [7]:
def output_messages(fitfile):
    """
    Go through the fitfile and extract time and heart rate data
    """
    messages = fitfile.get_messages()

    global current_timestamp
    global timestamp_16

    global hrdata
    global timestamp

    for record in messages:
    # Go through all the data entries in this record

        # Extract the global time stamp
        if record.name == 'monitoring_info':
            for r in record.fields:
                if r.field is not None:
                    if r.field.name == 'local_timestamp':
                        current_timestamp = r.value
                        timestamp_16 = None
                        #print("Timestamp: {}".format(current_timestamp))

        # get the heart rate data
        if record.name == 'monitoring':
            for r in record:
                if r.field is not None:
                    if r.field.name =='heart_rate':
                        time = get_time(record)
                        hr = get_heartrate(record)

                        timestamp.append(time)
                        hrdata.append(hr)
                        #print("Time: {}   Heart: {}".format(time,hr))

In [8]:
def output_csv(time, hr, fi):
    """
    Write the extracted time and heart rate data to csv fiile
    """
    data = [time, hr]

    csvFile = open(fi,'w')
    with csvFile:
        print("Writing csv to: {}".format(fi))
        writer = csv.writer(csvFile, lineterminator='\n') # need lineterminator to prevent extra linebreaks
        writer.writerow(["Date time", "Heart rate BPM"])
        for i in range(len(time)):
            writer.writerow([time[i],hr[i]])

In [15]:
#choose the right directory where fit files are located
dir = "data/Garmin_data/fit_data_20220503"
for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100028611307.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100028613810.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100049366702.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100065762999.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100083228457.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100083232525.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100120253803.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100167075335.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100184528596.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100184531902.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100258140755.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100269017811.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_100269021100.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail

data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_74941884069.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_74974332490.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_74974335252.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_74974763399.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75030216124.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75030221151.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75030221652.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75183279865.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75186035290.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75186036945.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75215018146.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75215018518.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75269909029.fit
data/Garmin_data/fit_data_20220503\higgyone@hotmail.com_75269912

In [29]:
# get the fit files in order (maybe)
files = sorted(Path(dir).glob('*.fit'))

In [31]:
files[0]

WindowsPath('data/Garmin_data/fit_data_20220503/higgyone@hotmail.com_100028611307.fit')

In [33]:
ff = dir+'/'+files[0].name
ff

'data/Garmin_data/fit_data_20220503/higgyone@hotmail.com_100028611307.fit'

In [36]:
fitfile = fp.FitFile(ff)
fitfile

In [47]:
mesx = fitfile.get_messages()
mesx

<generator object FitFile.get_messages at 0x000002D3C34BBCF0>

In [48]:
for r in mesx:
    print(r)
    for x in r:
        print(x)

file_id (#0)
garmin_product: 2158
manufacturer: garmin
number: 112
serial_number: 3984644549
time_created: 2021-07-24 02:34:00
type: monitoring_b
unknown_6: None
device_info (#23)
garmin_product: 2158
manufacturer: garmin
serial_number: 3984644549
software_version: 9.8
timestamp: 2021-07-24 02:34:00
software (#35)
version: 3.0
monitoring_info (#103)
activity_type: (6, 1)
cycles_to_calories: (0.047, 0.148) [kcal/cycle]
cycles_to_distance: (1.638, 2.457) [m/cycle]
local_timestamp: 2021-07-24 03:34:00
resting_metabolic_rate: 1964 [kcal/day]
timestamp: 2021-07-24 02:34:00
unknown_7: (11121, 11121)
unknown_8: None
monitoring (#55)
active_calories: 5 [kcal]
active_time: 180.0 [s]
activity_type: walking
distance: 47.5 [m]
duration_min: 214 [min]
steps: 58 [steps]
timestamp: 2021-07-24 02:34:00
monitoring (#55)
timestamp: 2021-07-24 02:34:00
unknown_37: 0
unknown_38: 0
unknown_211 (#211)
unknown_0: 46
unknown_1: 49
unknown_253: 996028440
unknown_24 (#24)
unknown_2: (144, 48, 94, 59, 1, 0, 48, 

unknown_0: 46
unknown_1: 50
unknown_253: 996057300


In [14]:
#choose the right directory hwere fit files are located
dir = "data/Fit_data/"
testdir = "C:/temp/data/test"
datadir = dir

UTC = pytz.UTC
GMT = pytz.timezone('Europe/London')

# heart rate data list
hrdata = []
# corresponding timestamp
timestamp = []

# globals to get the time data
current_timestamp = datetime.datetime.now()
timestamp_16 = None

In [ ]:
# get the fit files in order (maybe)
files = sorted(Path(datadir).glob('*.fit'))

for f in files:

    ff = datadir+'/'+f.name

    fitfile = fitparse.FitFile(ff)

    print("converting {}".format(f.name))

    output_messages(fitfile)

fi = datadir+'/'+'op.csv'
output_csv(timestamp, hrdata, fi)


In [26]:
np.uint16(np.iinfo(np.uint16).max) # unit16 max

65535